In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from skimage.data import imread
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/"))

# Any results you write to the current directory are saved as output.

In [ ]:
# ref: https://www.kaggle.com/paulorzp/run-length-encode-and-decode
def rle_decode(mask_rle, shape=(768, 768)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  # Needed to align to RLE direction

In [ ]:
masks = pd.read_csv("../input/train_ship_segmentations_v2.csv")

masks.head()

In [ ]:
is_boat = masks.EncodedPixels.notnull()
print('Found {} boats'.format(is_boat.sum()))
masks = masks[is_boat].reset_index().drop(['index'], axis=1)

In [ ]:
masks.head()

In [ ]:
for r, row in masks.iterrows():
    if r == 10: 
        break
    ImageId = masks.iloc[r]['ImageId']

    img = imread('../input/train_v2/' + ImageId)
    img_masks = masks.loc[masks['ImageId'] == ImageId, 'EncodedPixels'].tolist()

    # Take the individual ship masks and create a single mask array for all ships
    all_masks = np.zeros((768, 768))
    for mask in img_masks:
        all_masks += rle_decode(mask)

    fig, axarr = plt.subplots(1, 3, figsize=(15, 40))
    axarr[0].axis('off')
    axarr[1].axis('off')
    axarr[2].axis('off')
    axarr[0].imshow(img)
    axarr[1].imshow(all_masks)
    axarr[2].imshow(img)
    axarr[2].imshow(all_masks, alpha=0.4)
    plt.tight_layout(h_pad=0.1, w_pad=0.1)
    plt.show()


In [ ]:
def circular_hist(ax, x, bins=16, density=True, offset=0, gaps=True):
    """
    Produce a circular histogram of angles on ax.

    Parameters
    ----------
    ax : matplotlib.axes._subplots.PolarAxesSubplot
        axis instance created with subplot_kw=dict(projection='polar').

    x : array
        Angles to plot, expected in units of radians.

    bins : int, optional
        Defines the number of equal-width bins in the range. The default is 16.

    density : bool, optional
        If True plot frequency proportional to area. If False plot frequency
        proportional to radius. The default is True.

    offset : float, optional
        Sets the offset for the location of the 0 direction in units of
        radians. The default is 0.

    gaps : bool, optional
        Whether to allow gaps between bins. When gaps = False the bins are
        forced to partition the entire [-pi, pi] range. The default is True.

    Returns
    -------
    n : array or list of arrays
        The number of values in each bin.

    bins : array
        The edges of the bins.

    patches : `.BarContainer` or list of a single `.Polygon`
        Container of individual artists used to create the histogram
        or list of such containers if there are multiple input datasets.
    """
    # Wrap angles to [-pi, pi)
    x = (x+np.pi) % (2*np.pi) - np.pi

    # Force bins to partition entire circle
    if not gaps:
        bins = np.linspace(-np.pi, np.pi, num=bins+1)

    # Bin data and record counts
    n, bins = np.histogram(x, bins=bins)

    # Compute width of each bin
    widths = np.diff(bins)

    # By default plot frequency proportional to area
    if density:
        # Area to assign each bin
        area = n / x.size
        # Calculate corresponding bin radius
        radius = (area/np.pi) ** .5
    # Otherwise plot frequency proportional to radius
    else:
        radius = n

    # Plot data on ax
    patches = ax.bar(bins[:-1], radius, zorder=1, align='edge', width=widths)

    # Set the direction of the zero angle
    ax.set_theta_offset(offset)

    # Remove ylabels for area plots (they are mostly obstructive)
    if density:
        ax.set_yticks([])

    return n, bins, patches

In [ ]:
def azimuth(point1, point2):
    '''azimuth between 2 shapely points (interval 0 - 360)'''
    
    angle = np.arctan2(point2[0] - point1[0], point2[1] - point1[1])
    return np.degrees(angle) if angle >= 0 else np.degrees(angle) + 360

def get_angle(point1, point2):
    degrees = azimuth(point1, point2)
    return degrees * np.pi / 180

def distance(point1, point2):
    return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

In [ ]:
!pip install rasterio fiona shapely geopandas

In [ ]:
import rasterio
import shapely.geometry
import rasterio.features
from shapely.geometry import Polygon
import glob

In [ ]:
xs = []
for r,row in masks.iterrows():
     
    img = (np.squeeze(rle_decode(masks.iloc[r]['EncodedPixels'])))
    img = (img > 0).astype('int16')
    shapes = rasterio.features.shapes(img)
    polygons = [shapely.geometry.Polygon(s[0]["coordinates"][0]) for s in shapes if s[1] == 1]
    sole_polygon = polygons[0]
    
    rect = sole_polygon.minimum_rotated_rectangle
    points = list(rect.exterior.coords)
    max_dist = 0
    max_idx = None
    
    dist_arr = []
    for i in range(4):
        dist = distance(points[i], points[i+1])
        dist_arr.append(dist)
        if dist >= max_dist:
            max_dist = dist
            max_idx = i
            
    for i in range(4):
        dist = distance(points[i], points[i+1])
        if dist == max_dist:
            point1 = points[i]
            point2 = points[i+1]
            radians = get_angle(point1, point2)
            xs.append(radians)

In [ ]:
offset=360-36
offset_r = 350

fig, ax = plt.subplots(1,1,figsize=(8,8), subplot_kw=dict(projection='polar'))

hist, bin_edges, patches = circular_hist(ax, np.array(xs), density=False, bins=60, gaps=True)
#plt.plot([(180+offset)*np.pi/180,offset*np.pi/180], [offset_r,offset_r], color="#ccc", linewidth=0.5, zorder=1)

ax.set_rlabel_position(offset)

plt.title(name)
plt.yticks([100,200,300], fontsize=14, backgroundcolor="#cccc")
plt.tick_params(axis="y", zorder=3)
plt.xticks([0,np.pi/2,np.pi, 3*np.pi/2], fontsize=15)

ax.grid(b=True, which='major', axis='x', color='#000', linestyle='-', linewidth=1)
ax.grid(b=False, which='minor', axis='x', color='#000', linestyle='-', linewidth=1)
ax.grid(b=True, which='major', axis='y', color='#ccc', linestyle='-', linewidth=0.6, zorder=0)
ax.grid(b=False, which='minor', axis='y')

ax.spines['polar'].set_visible(False)

plt.show()
plt.close()

In [ ]:
len(xs)

In [ ]:
len(xs2)

In [ ]:
len(masks)

In [ ]:
asr_arr = []
for r,row in masks.iterrows():
    img = (np.squeeze(rle_decode(masks.iloc[r]['EncodedPixels'])))
    img = (img > 0).astype('int16')
    shapes = rasterio.features.shapes(img)
    polygons = [shapely.geometry.Polygon(s[0]["coordinates"][0]) for s in shapes if s[1] == 1]
    sole_polygon = polygons[0]
    
    rect = sole_polygon.minimum_rotated_rectangle
    points = list(rect.exterior.coords)
    dist_arr = []
    for i in range(4):
        dist = distance(points[i], points[i+1])
        dist_arr.append(dist)
        if dist >= max_dist:
            max_dist = dist
            max_idx = i
    wh = (sorted(np.unique(dist_arr)))
    if (len(wh) == 1):
        asr_arr.append(1.0)
    else:
        asr = (wh[1] / wh[0])
        asr_arr.append(asr)


In [ ]:
fig, ax = plt.subplots(1,1)
ax.hist(np.array(asr_arr), bins=np.logspace(0,1,num=20))
ax.set_xscale("log")
ax.set_xlabel('Aspect Ratio')
ax.set_ylabel('Frequency')

In [ ]:
asr_arr = np.array(asr_arr)
len(asr_arr)
len(asr_arr[asr_arr < 1.01]) / len(asr_arr)

In [ ]:
count = 0
for i in asr_arr:
    if i < 1.1:
        count+=1
        
print(count / len(asr_arr))

In [ ]:
xs2 = xs